In [1]:
import warnings
warnings.filterwarnings("ignore")

# Import Torch, MONAI and other libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Compose, Normalize, ToTensor
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from monai.transforms import Compose, ScaleIntensity, EnsureChannelFirst, Resize, EnsureChannelFirstd
from monai.networks.layers import Norm
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set random seed
torch.manual_seed(0)

# Import custom classes/functions
from cg_Dataset import load_data

In [2]:
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [13]:
# Transforms
transforms_img = Compose([
    # Scale the intensity of the image data
    # ScaleIntensity(),
    EnsureChannelFirst(channel_dim=-1),
    
    # Resize the image data to a uniform size, 128x128
    Resize(spatial_size=(128, 128)),
    
    # Normalize the image data
    #Normalize(mean=0.0, std=1.0),
    
    # Convert the image data to a tensor
    # ToTensor(),
])

transforms_mask = Compose([
    # Scale the intensity of the image data
    # ScaleIntensity(),
    
    # Resize the image data to a uniform size, 128x128
    Resize(spatial_size=(128, 128)),
    
    # Normalize the image data
    #Normalize(mean=0.0, std=1.0),
    
    # Convert the image data to a tensor
    # ToTensor(),
])

transforms = [transforms_img, transforms_mask]

In [14]:
# Load data
train_data = load_data("train", asImage = False, filecount = 0, transform = transforms)
val_data = load_data("val", asImage = False, filecount = 0, transform = transforms)

# Print the length of the training and validation data
print(f"Train data: {len(train_data)}")
print(f"Val data: {len(val_data)}")

Train data: 19535
Val data: 2653


In [17]:
# Create a DataLoader for the training and validation data
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False)